In [ ]:
import pandas as pd

# Steps to open a csv file in google colab
from google.colab import files
news_data = files.upload()

Saving fake_or_real_news.csv to fake_or_real_news.csv


In [ ]:
import io
df1 = pd.read_csv(io.BytesIO(news_data['fake_or_real_news.csv']))

In [ ]:
x = df1['text']
y = df1['label']

In [ ]:
df1.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df1['label'].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [ ]:
import numpy as np

In [ ]:
# def split_train_test(data,test_ratio):
#   np.random.seed(42)       
#   #========> We used np.random.seed to avoid the problem of overfitting our data i.e we fixed the shuffled numbers to show same output while we perform shuffle
#   shuffled = np.random.permutation(len(data))
#   print(shuffled)
#   test_set_size = int(len(data)*test_ratio)
#   test_indices = shuffled[:test_set_size]
#   train_indices = shuffled[test_set_size:]
#   return data.iloc[train_indices],data.iloc[test_indices]

In [ ]:
# train_set,test_set = split_train_test(df1,0.2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from sklearn.metrics import classification_report

x_train , x_test, y_train, y_test = train_test_split(x,y,test_size =0.2,random_state =42)

In [ ]:
x_train.shape

(5068,)

In [ ]:
x_test.shape

(1267,)

In [ ]:
y_test.shape

(1267,)

In [ ]:
y_train.shape

(5068,)

In [ ]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)


In [ ]:
tfvect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.7, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
tfid_x_train = tfvect.fit_transform(x_train)

In [ ]:
tfid_x_test = tfvect.transform(x_test)

In [ ]:
tfid_x_train.shape

(5068, 61359)

In [ ]:
tfid_x_test.shape

(1267, 61359)

##PassiveAggressiveClassifier

In [ ]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(tfid_x_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
pac_pred = classifier.predict(tfid_x_test)


In [ ]:
score = accuracy_score(y_test,pac_pred)
print(f"Accuracy:{round(score*100,2)}%")

Accuracy:93.84%


In [ ]:
cf = confusion_matrix(y_test,pac_pred,labels=['FAKE','REAL'])

In [ ]:
print(cf)

[[590  38]
 [ 40 599]]


In [ ]:
print(classification_report(y_test,pac_pred))

              precision    recall  f1-score   support

        FAKE       0.94      0.94      0.94       628
        REAL       0.94      0.94      0.94       639

    accuracy                           0.94      1267
   macro avg       0.94      0.94      0.94      1267
weighted avg       0.94      0.94      0.94      1267



In [ ]:
def fake_news_det(news):
  input_data = [news]
  vectorized_input_data = tfvect.transform(input_data)
  prediction = classifier.predict(vectorized_input_data)
  print(prediction)

In [ ]:
fake_news_det("Trump takes on Cruz,but lightly")

['REAL']


## Random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(tfid_x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
RFC_pred = RFC.predict(tfid_x_test)

In [ ]:
RFC_score = accuracy_score(y_test,RFC_pred)
print(f"Accuracy:{round(RFC_score*100,2)}%")

Accuracy:91.32%


In [ ]:
print(classification_report(y_test,RFC_pred))

              precision    recall  f1-score   support

        FAKE       0.92      0.90      0.91       628
        REAL       0.90      0.93      0.91       639

    accuracy                           0.91      1267
   macro avg       0.91      0.91      0.91      1267
weighted avg       0.91      0.91      0.91      1267



## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier()
DT.fit(tfid_x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
DT.score(tfid_x_test,y_test)

0.8161010260457774

In [ ]:
pred_DT = DT.predict(tfid_x_test)

In [ ]:
DT_score = accuracy_score(y_test,pred_DT)
print(f"Accuracy:{round(DT_score*100,2)}%")

Accuracy:81.61%


In [ ]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

        FAKE       0.82      0.81      0.81       628
        REAL       0.82      0.82      0.82       639

    accuracy                           0.82      1267
   macro avg       0.82      0.82      0.82      1267
weighted avg       0.82      0.82      0.82      1267



## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(tfid_x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
LR.score(tfid_x_test,y_test)

0.9155485398579322

In [ ]:
pred_LR = LR.predict(tfid_x_test)

In [ ]:
LR_score = accuracy_score(y_test,pred_LR)
print(f"Accuracy:{round(LR_score*100,2)}%")

Accuracy:91.55%


In [ ]:
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

        FAKE       0.90      0.93      0.92       628
        REAL       0.93      0.90      0.91       639

    accuracy                           0.92      1267
   macro avg       0.92      0.92      0.92      1267
weighted avg       0.92      0.92      0.92      1267



## LightBGM classifier

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_model = lgb.LGBMClassifier(learning_rate=0.5,max_depth=-5,random_state=42)
lgb_model.fit(tfid_x_train,y_train,eval_set=[(tfid_x_test,y_test),(tfid_x_train,y_train)],verbose = 20,eval_metric = 'logloss')

[20]	training's binary_logloss: 0.0337908	training's binary_logloss: 0.0337908	valid_0's binary_logloss: 0.197857	valid_0's binary_logloss: 0.197857
[40]	training's binary_logloss: 0.00435615	training's binary_logloss: 0.00435615	valid_0's binary_logloss: 0.185943	valid_0's binary_logloss: 0.185943
[60]	training's binary_logloss: 0.000630581	training's binary_logloss: 0.000630581	valid_0's binary_logloss: 0.204626	valid_0's binary_logloss: 0.204626
[80]	training's binary_logloss: 9.31633e-05	training's binary_logloss: 9.31633e-05	valid_0's binary_logloss: 0.222486	valid_0's binary_logloss: 0.222486
[100]	training's binary_logloss: 1.44184e-05	training's binary_logloss: 1.44184e-05	valid_0's binary_logloss: 0.248767	valid_0's binary_logloss: 0.248767


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.5, max_depth=-5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
print("Training Accuracy={:.4f}".format(lgb_model.score(tfid_x_train,y_train)))
print("Testing Accuracy={:.4f}".format(lgb_model.score(tfid_x_test,y_test)))

Training Accuracy=1.0000
Testing Accuracy=0.9392


In [ ]:
print(classification_report(y_test,lgb_model.predict(tfid_x_test)))

              precision    recall  f1-score   support

        FAKE       0.94      0.94      0.94       628
        REAL       0.94      0.94      0.94       639

    accuracy                           0.94      1267
   macro avg       0.94      0.94      0.94      1267
weighted avg       0.94      0.94      0.94      1267

